# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install openai

In [1]:
from keys import *
from openai import OpenAI
from groq import Groq

client = Groq(api_key=groq_key)

In [2]:
llm_model = "llama3-8b-8192"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [6]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=1,
        max_tokens=1024,
        top_p=1,
    )
    print(chat_completion)
    return chat_completion.choices[0].message.content


In [7]:
get_completion("What is 1+1?")

ChatCompletion(id='chatcmpl-441af4c2-d8cd-48e8-9b23-43573cf21bb7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The answer to 1+1 is 2.', role='assistant', function_call=None, tool_calls=None))], created=1722760930, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_873a560973', usage=CompletionUsage(completion_tokens=12, prompt_tokens=17, total_tokens=29, completion_time=0.008854077, prompt_time=0.00329279, queue_time=None, total_time=0.012146867), x_groq={'id': 'req_01j4e8v8saewfv9h0nywp4stfv'})


'The answer to 1+1 is 2.'

In [8]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [9]:
style = """American English \
in a calm and respectful tone
"""

In [10]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [11]:
response = get_completion(prompt)

ChatCompletion(id='chatcmpl-bed1a3c3-b225-4132-af7e-8d9f32464eeb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here is the translated text in American English, written in a calm and respectful tone:\n\n"I\'m really frustrated that my blender lid came loose and splattered my kitchen walls with smoothie. And to make matters worse, the warranty doesn\'t cover the cost of cleaning up my kitchen. Could you please help me with this issue?"', role='assistant', function_call=None, tool_calls=None))], created=1722760971, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_873a560973', usage=CompletionUsage(completion_tokens=66, prompt_tokens=92, total_tokens=158, completion_time=0.052197999, prompt_time=0.0187721, queue_time=None, total_time=0.07097009900000001), x_groq={'id': 'req_01j4e8wgx2et0vfkyz2jgw3hkg'})


In [12]:
response

'Here is the translated text in American English, written in a calm and respectful tone:\n\n"I\'m really frustrated that my blender lid came loose and splattered my kitchen walls with smoothie. And to make matters worse, the warranty doesn\'t cover the cost of cleaning up my kitchen. Could you please help me with this issue?"'

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#!pip install --upgrade langchain

### Model

In [1]:
from langchain_experimental.llms import ChatLlamaAPI
from llamaapi import LlamaAPI
from keys import *

llama = LlamaAPI(llama_api_key)

In [2]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatLlamaAPI(client=llama)
chat

ChatLlamaAPI(client=<llamaapi.llamaapi.LlamaAPI object at 0x106bf5160>)

### Prompt template

In [3]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [4]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [5]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [6]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [8]:
customer_style = """American English \
in a calm and respectful tone
"""

In [9]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [10]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [11]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [12]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [13]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

/Users/amirrezaalasti/Documents/i3center_datascience/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [14]:
print(customer_response.content)

Here is the translated text in American English, written in a calm and respectful tone:

"I'm extremely frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I really need your help right now."

Let me know if you have any further requests!


In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatLlamaAPI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

In [ ]:
type(response.content)

In [ ]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

### Parse the LLM output string into a Python dictionary

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [ ]:
response = chat(messages)

In [ ]:
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('delivery_days')